In [1]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
import json
import help_functions as hf
import numpy as np
import time
hf.setup_gpu(gpu_nr=1)

AttributeError: module 'tensorflow' has no attribute 'setup_gpu'

In [3]:
# train_output = (1 - labels) * get_constr_out(output, M) + labels * get_constr_out(labels * output, M)

# TODO: add an MCM (maximum constraint module) layer here. Hierarchy constraint expressed in matrix R
# it seems the MCM layer only is used at inference? No

In [111]:
with open('training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(1)]

config['batch_size'] = 1
config['nr_classes'] = 3
test = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                   nr_classes=config['nr_classes'],
                   batch_size=config['batch_size'],
                   image_dimension=config['image_dimension'])

Found 50000 non-validated image filenames belonging to 40 classes.


/home/matvieir/wiki_image_classification/src/classification/help_functions.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 37088 validated image filenames belonging to 3 classes.


In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):

        print(f'len(data): {len(data)}')

        inputs, labels = data

        with tf.GradientTape() as tape:
            y_probs = model(inputs, training=True)

            # Extra steps for coherent HMC:
            # 1. max constraint module        
            term_1 = (1 - labels) * max_constrain(y_probs, mask)
            term_2 = labels * max_constrain(labels * y_probs, mask)
            y_probs_constrained = term_1 + term_2

            # 2. modify what is sent to binary cross-entropy function (not anymore y_true and y_probs)
            loss_value = model.compiled_loss(labels, y_probs_constrained)
            # loss_value = loss_fn(labels, y_probs_constrained)

        grads = tape.gradient(loss_value, model.trainable_weights)
        # optimizer.apply_gradients(zip(grads, model.trainable_weights))
        model.optimizer.apply_gradients(zip(grads, model.trainable_weights))
        model.metrics.update_state(labels, y_probs_constrained)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

def get_uncompiled_model():
    basemodel = EfficientNetB2(include_top=False, weights=None, classes=20, input_shape=(32, 32, 3))
    inputs = Input(shape=(32, 32, 3))
    x = basemodel(inputs)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(20, activation='sigmoid')(x)
    model = CustomModel(inputs=inputs, outputs=outputs)
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'], loss='binary_crossentropy')
    return model

model = get_compiled_model()
model.fit(test, verbose=1, epochs=1)

In [112]:
basemodel = EfficientNetB2(include_top=False, weights=None, classes=config['nr_classes'], input_shape=(config['image_dimension'], config['image_dimension'], 3))
inputs = Input(shape=(config['image_dimension'], config['image_dimension'], 3))
x = basemodel(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(config['nr_classes'], activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['categorical_accuracy'], loss='binary_crossentropy')
# loss_fn = tf.keras.losses.BinaryCrossentropy()
# optimizer = tf.keras.optimizers.Adam()
# metrics = tf.metrics.CategoricalAccuracy()

In [88]:
# M_ij = 1 if the class j is a subclass of the class i. I.e.: row i, column j is 1.
# - note that if M_ij is 1 then M_ji is necessarily 0 as the subclass relation is not reflexive
# - note that M_ij is also 1 if j is not a direct descendant of i 

In [114]:
mask = np.array([[1, 0, 1, 1, 0],
                 [0, 1, 0, 0, 1],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1],], dtype=np.float32)

In [119]:
def max_constrain(output, mask):
    "Constrains the output given the hierarchy expressed by the mask."

    batch_size = len(output) # or output.shape[0]
    nr_classes = mask.shape[0] # = mask.shape[1] = output.shape[1]

    output = tf.expand_dims(output, axis=1) # Pytorch unsqueeze()

    batch_output = tf.broadcast_to(output, [batch_size, nr_classes, nr_classes]) # this is H in the MCM equation
    batch_mask   = tf.broadcast_to(mask,   [batch_size, nr_classes, nr_classes])

    constrained_output = tf.math.reduce_max(batch_output * batch_mask, axis=2)

    return constrained_output

# @tf.function   # to speed up: https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch#speeding-up_your_training_step_with_tffunction
def train_step(inputs, labels):

    print(f'INPUTS: \n{inputs}')
    print(f'LABELS: \n{labels}')

    with tf.GradientTape() as tape:
        y_probs = model(inputs, training=True)

        # Extra steps for coherent HMC:
        # 1. max constraint module        
        term_1 = (1 - labels) * max_constrain(y_probs, mask)
        term_2 = labels * max_constrain(labels * y_probs, mask)
        y_probs_constrained = term_1 + term_2

        # 2. modify what is sent to binary cross-entropy function
        loss_value = model.compiled_loss(labels, y_probs_constrained)
        # loss_value = loss_fn(labels, y_probs_constrained)

    grads = tape.gradient(loss_value, model.trainable_weights)
    # optimizer.apply_gradients(zip(grads, model.trainable_weights))
    model.optimizer.apply_gradients(zip(grads, model.trainable_weights))
    model.metrics.update_state(labels, y_probs_constrained)
    return loss_value

In [ ]:
epochs = 1
for epoch in range(epochs):
    print(f'Start of epoch {epoch}\n')
    end = time.time()
    # Iterate over the batches of the dataset
    for step, (x_batch_train, y_batch_train) in enumerate(test):
        loss_value = train_step(x_batch_train, y_batch_train)
        print(f'LOSS_VALUE: \n{loss_value}')

        # Log every 200 batches.
        if step % 300 == 0:
            print(f"Training loss (for one batch) at step {step}: {float(loss_value):.4f}")
            print(f"Seen so far: {(step + 1) * config['batch_size']} samples")
            temp_end = time.time()
            print(f'time: {temp_end-end}')
            end = time.time()

    # Display metrics at the end of each epoch.
    train_acc = model.metrics.result()
    print(f"Training acc over epoch: {float(train_acc)}")

    # Reset training metrics at the end of each epoch
    model.metrics.reset_states()

#### Unit tests of the Maximum Constraint Module

In [ ]:
# Unit test of maximum constraint module: 3 classes
# Class 1 is the parent of classes 2 and 3
mask = np.array([[1, 0, 0],
                 [1, 1, 0],
                 [1, 0, 1]], dtype=np.float32)
np.fill_diagonal(mask, 1)
mask = np.transpose(mask)

# Case 1: the parent has greater probability than both children. Nothing changes.
probs = np.array([[0.7, 0.5, 0.2]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == probs).all())

# Case 2: the parent has smaller probability than child class 2. 
probs = np.array([[0.1, 0.5, 0.2]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.5, 0.5, 0.2], dtype=np.float32)).all())

# Case 3: the parent has smaller probability than child class 3. 
probs = np.array([[0.1, 0.3, 0.8]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.8, 0.3, 0.8], dtype=np.float32)).all())

# Case 4: the parent has smaller probability than both children. 
probs = np.array([[0.2, 0.3, 0.5]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.5, 0.3, 0.5], dtype=np.float32)).all())

In [129]:
# Unit test of maximum constraint module: 5 classes
#     1  2
#    / \  \
#   3   4  5
# Class 1 is the parent of classes 3 and 4; class 2 is the parent of class 5
mask = np.array([[1, 0, 1, 1, 0],
                 [0, 1, 0, 0, 1],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1],], dtype=np.float32)

# Case 1: all is good (1 has great prob than 3 and 4; 2 has greater prob than 5)
probs = np.array([[0.7, 0.5, 0.2, 0.1, 0.1]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == probs).all())

# Case 2: 1 has smaller prob than 3; 2 is ok
probs = np.array([[0.7, 0.5, 0.9, 0.1, 0.1]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.9, 0.5, 0.9, 0.1, 0.1], dtype=np.float32)).all())

# Case 3: 1 has smaller prob than 4; 2 is ok
probs = np.array([[0.7, 0.5, 0.6, 0.9, 0.1]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.9, 0.5, 0.6, 0.9, 0.1], dtype=np.float32)).all())

# Case 4: 1 has smaller prob than 4; 2 has smaller prob than 5
probs = np.array([[0.7, 0.5, 0.6, 0.9, 0.8]], dtype=np.float32)
assert((max_constrain(probs, mask)[0].numpy() == np.array([0.9, 0.8, 0.6, 0.9, 0.8], dtype=np.float32)).all())

tf.Tensor([[0.7 0.5 0.2 0.1 0.1]], shape=(1, 5), dtype=float32)


In [94]:
def get_parent_child_dict(mask):
    dict = {}
    for row_idx in range(mask.shape[0]):
        children_idx = np.nonzero(mask[row_idx, :])[0]
        if children_idx.size:
            dict[row_idx] = children_idx
    return dict

def is_coherent(probs, parent_idx, children_idx):
    for child_idx in children_idx:
        assert(probs[parent_idx] >= probs[child_idx])

def coherence_test(mask, nr_tests=100):
    parent_child_dict = get_parent_child_dict(mask)
    for _ in range(nr_tests):
        probs = np.random.uniform(low=0.0, high=1.0, size=(1, mask.shape[0]))
        probs = np.array(probs, dtype=np.float32)
        coherent_probs = max_constrain(probs, mask)[0] # '0' because the output of max_constrain is a copy of the same coherent probs in several lines
        for parent_idx in parent_child_dict:
            is_coherent(coherent_probs, parent_idx, parent_child_dict[parent_idx])  
    print('Test passed, all probs are coherent!')

Trees with 2 layers

In [ ]:
#   1   2
#  / \   \
# 3   4   5
mask = np.array([[1, 0, 1, 1, 0],
                 [0, 1, 0, 0, 1],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1],], dtype=np.float32)

coherence_test(mask)    

#     1
#  / / \ \
# 2 3  4  5
mask = np.array([[1, 1, 1, 1, 1],
                 [0, 1, 0, 0, 0],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1],], dtype=np.float32)

coherence_test(mask)   

#  1        5     8 
# / \ \    / \   / \
# 2 3  4  6   7  9  10
mask = np.array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=np.float32)

coherence_test(mask) 

Trees with 3 or more layers

In [ ]:
# 1
# |
# 2
# |
# 3
mask = np.array([[1, 1, 1],
                 [0, 1, 1],
                 [0, 0, 1]], dtype=np.float32)
coherence_test(mask)

#     1    5
#    / \   |
#   2   3  6
#  /       |
# 4        7
mask = np.array([[1, 1, 1, 1, 0, 0, 0],
                 [0, 1, 0, 1, 0, 0, 0],
                 [0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 1, 1, 1],
                 [0, 0, 0, 0, 0, 1, 1],
                 [0, 0, 0, 0, 0, 0, 1],], dtype=np.float32)
coherence_test(mask)

#### Unit tests of the loss function
Compare the result with the official PyTorch implementation. Indeed the same result!

In [143]:
mask = np.array([[1, 0, 1, 1, 0],
                 [0, 1, 0, 0, 1],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1],], dtype=np.float32)
                 
# y_probs = np.array([[0.35, 0.03, 0.6, 0.08, 0.1],
#                     [0.9, 0.03, 0.05, 0.98, 0.01],
#                     [0.57, 0.33, 0.7, 0.2, 0.12]], dtype=np.float32)

y_probs = np.array([[0.35, 0.03, 0.6, 0.08, 0.1]], dtype=np.float32)

labels = np.array([[0, 0, 1, 0, 1]], dtype=np.float32)


term_1 = (1 - labels) * max_constrain(y_probs, mask)
term_2 = labels * max_constrain(labels * y_probs, mask)
y_probs_constrained = term_1 + term_2

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(labels, y_probs_constrained).numpy()

0.7836884